In [8]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0 
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello
y_one_hot = [[[0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 0, 1]]]  # o 4

inp = tf.random_normal([3, 4])
attns = tf.random_normal([4, 2])
tf.contrib.layers.fully_connected(tf.matmul(inp,attns),10)

<tf.Tensor 'fully_connected/Relu:0' shape=(3, 10) dtype=float32>

In [8]:
input_dim = 5  # one-hot size
hidden_size = 10  # output from the LSTM. 5 to directly predict one-hot
output_dim = 5
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.float32, [None, sequence_length, output_dim])  # Y one-hot

cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
#cell = tf.contrib.rnn.GRUCell(num_units=hidden_size)
#cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)

outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

print(outputs, _states, cell.output_size, cell.state_size)

w = tf.Variable(tf.random_normal([hidden_size, output_dim]))
b = tf.Variable(tf.random_normal([output_dim]))

outputs = tf.reshape(outputs, [-1, hidden_size])
y_pred = tf.nn.softmax(tf.matmul(outputs,w)+b)
y_pred = tf.reshape(y_pred, [-1, sequence_length, output_dim])
print('y_pred', y_pred)

y_label_pred = tf.argmax(y_pred, 2)


Tensor("rnn/transpose:0", shape=(1, 6, 10), dtype=float32) Tensor("rnn/while/Exit_2:0", shape=(1, 10), dtype=float32) 10 10
y_pred Tensor("Reshape_1:0", shape=(1, 6, 5), dtype=float32)


In [3]:
loss = -tf.reduce_sum(Y*tf.log(y_pred), name = 'cross_entropy')
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [17]:
feed_dict = {X:x_one_hot, Y:y_one_hot}
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(1000):
    l, _ = sess.run([loss,train], feed_dict=feed_dict)
    if i% 100==0:
#         print('lost:', l)
        predy, realy = sess.run([y_label_pred, y_pred], feed_dict={X:x_one_hot})
        print('lost:', l, 'pred :',predy, 'real_y',y_one_hot)
        
        result_str = [idx2char[c] for c in np.squeeze(predy)]
        print( " prediction str: " ,''.join(result_str))
        
    


lost: 13.7961 pred : [[4 0 2 3 3 4]] real_y [[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]]
 prediction str:  ohello
lost: 0.00132686 pred : [[1 0 2 3 3 4]] real_y [[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]]
 prediction str:  ihello
lost: 0.000774916 pred : [[1 0 2 3 3 4]] real_y [[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]]
 prediction str:  ihello
lost: 0.000507617 pred : [[1 0 2 3 3 4]] real_y [[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]]
 prediction str:  ihello
lost: 0.000357997 pred : [[1 0 2 3 3 4]] real_y [[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]]
 prediction str:  ihello
lost: 0.000265366 pred : [[1 0 2 3 3 4]] real_y [[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0